In [1]:
import pandas as pd

In [8]:
header = ['DESTINATION','CORE_TIME','POLLER_TIME','TARGET_ID',
           'POLICY_ID','POLICY_TYPE','DETECTED_SEVERITY','PROPAGATED_SEVERITY',
           'LOW_THRESHOLD','MEASURED_VALUE','HIGH_THRESHOLD','PROPAGATED_FLAG','EVENT_CODE',
           'METRIC_NAME','RECON_ID','BUS_ID','PARENT_BUS_ID','NEW_BUS_ID','RECONCILIATION_NUM',
           'DEVICE_TYPE','OPERATING_SYSTEM','RECONCILIATION_ID','SITE','TYPE_NAME',
           'IS_VIRTUAL','PARENT_TARGET_ID','MONITOR_DESC', 'CRAP']
events = pd.read_csv("event-logs-merged.csv", names = header)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
events = events[['CORE_TIME','RECONCILIATION_ID']]

In [ ]:
events.shape

In [ ]:
events.iloc[0]

In [ ]:
# print(events['DEVICE_TYPE'].unique())
# print(events['TYPE_NAME'].unique())
DEVICE_TYPE = events['DEVICE_TYPE'].unique()
MONITOR_DESC = events['MONITOR_DESC'].unique()
print(DEVICE_TYPE)
print(len(MONITOR_DESC))

In [ ]:
X = events[['MONITOR_DESC','DEVICE_TYPE','SITE','NEW_BUS_ID','IS_VIRTUAL']]
# X.loc[:'NEW_BUS_ID'] = X.loc[:'NEW_BUS_ID'].astype(str)
X['SITE'] = X['SITE'].str.replace(' ','').str.lower()
X['SITE'] = X['SITE'].apply(lambda x: '400minutemanrd' if ('400' in str(x)) else x)

X['DEVICE_TYPE'] = X['DEVICE_TYPE'].str.replace(' ','').str.lower()
X['NEW_BUS_ID'] = X['NEW_BUS_ID'].astype(str)
Y = events['DETECTED_SEVERITY']
X = pd.get_dummies(data=X, drop_first=True)

from sklearn import linear_model
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .20, random_state = 40)
regr = linear_model.LinearRegression()
regr.fit(X_train, Y_train)
predicted = regr.predict(X_test)

In [ ]:
coef = pd.DataFrame(list(regr.coef_),list(X_train.columns))
coef = coef.reset_index()
coef.columns = ['dummy_name', 'coef']
pd.options.display.float_format = '{:,.2f}'.format
print(coef[coef['dummy_name'].str.contains('MONITOR_DESC')])
print(coef[coef['dummy_name'].str.contains('DEVICE_TYPE')])
print(coef[coef['dummy_name'].str.contains('SITE')])
print(coef[coef['dummy_name'].str.contains('NEW_BUS_ID')])
print(coef[coef['dummy_name'].str.contains('IS_VIRTUAL')])

In [10]:
def cluster_events(events, time_step_buffer):
    event_clusters = []
    event_clusters_len = []
    current_cluster = []
    current_recon_id = None
    current_core_time = None

    for index, event in events.iterrows():
        if (len(current_cluster) == 0):
            current_recon_id = event["RECONCILIATION_ID"]
            current_core_time = event["CORE_TIME"]
            current_cluster.append(event)
        elif (event["RECONCILIATION_ID"] == current_recon_id and (event["CORE_TIME"] - current_core_time) <= time_step_buffer):
            current_cluster.append(event)
        else:
            event_clusters.append(current_cluster)
            event_clusters_len.append(len(current_cluster))
            current_cluster = []
            current_recon_id = None
            current_core_time = None
    return event_clusters, event_clusters_len

In [11]:
event_clusters_10, event_clusters_10_len = cluster_events(events,10)
print(pd.DataFrame(event_clusters_10_len).describe())
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
plt.hist(list(filter(lambda a: a > 10, event_clusters_10_len)), bins=range(0, 200, 5))
plt.show()

KeyboardInterrupt: 

In [ ]:
del event_clusters_5[:]
del event_clusters_5

del event_clusters_10[:]
del event_clusters_10

In [12]:
event_clusters_5, event_clusters_5_len = cluster_events(events,5)
print(pd.DataFrame(event_clusters_5).describe())
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
plt.hist(list(filter(lambda a: a > 10, event_clusters_5)), bins=range(0, 200, 5))
plt.show()

MemoryError: 